In [22]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from jupyter_datatables import init_datatables_mode

In [23]:
def multiplier(EF, concat, colname,df_vocs):
    df3 = voc_concat.merge(df_vocs,left_on=["Modo","Motorizacion","Norma"],right_on=["Modo","Motorizacion","Norma"],how="left")
    multiple = df3.loc[:,'2014':2060].multiply(df3[colname], axis="index")/100
    multiple = pd.concat([df3.iloc[:,0:6], multiple], axis=1)
    multiple["Pollutant"] = colname
    return multiple
    

In [24]:
#cargando. excels
#xls = pd.ExcelFile('Copert5.xlsx')
xls = pd.ExcelFile('1.A.3.b.i-iv Road Transport Appendix 4 Emission Factors 2021.xlsx')
df = pd.read_excel(xls,'HOT_EMISSIONS_PARAMETERS', index_col=None)

In [25]:
#Filtrando usando REGEX a las categorías vehiculares a utilizar
df = df.loc[
      ((df['Fuel'] == 'Petrol') |  (df['Fuel'] == 'Diesel')) &
      df.Segment.str.contains('^Medium') | #VLP, TAX
      df.Segment.str.contains("^N1-II$")  | #VLC
      df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*') | #CAM PES RIGID>28 t and articulated
      df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *') |#CAM MED RIGID 14-28 t
      df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *') |#CAM LIV
      df.Segment.str.contains('^Urban Buses Standard 15 *') | #Bus RIG
      df.Segment.str.contains('^Urban Buses Articulated >18 t') | #BUS ART
      df.Segment.str.contains('^Coaches Standard <=18 t') | #BUS INT PROV
      df.Segment.str.contains("^Motorcycles 4-stroke *") #MOT revisar
]

In [26]:
#Cambiando a la clasificacion a utilizar
df.loc[df['Segment'] == 'Medium', 'Segment'] = 'VLP'
df.loc[df['Segment'] == 'N1-II', 'Segment'] = 'VLC'
df.loc[df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *'), 'Segment'] = 'CAM LIV'
df.loc[df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *'), 'Segment'] = 'CAM MED'
df.loc[df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*'), 'Segment'] = 'CAM PES'
df.loc[df.Segment.str.contains('^Urban Buses Standard 15 *'), 'Segment'] = 'BUS RIG'
df.loc[df.Segment.str.contains('^Urban Buses Articulated >18 t'), 'Segment'] = 'BUS ART'
df.loc[df.Segment.str.contains('^Coaches Standard <=18 t'), 'Segment'] = 'BUS INT PROV'
df.loc[df.Segment.str.contains("^Motorcycles 4-stroke *"), 'Segment'] = 'MOT'
#df.to_excel('C5_reducido.xlsx',index = False)
#df.loc[df.Segment.str.contains('^CAM MED*')] 

In [27]:
df = df.loc[ ((df['Load'].isnull())) | #todas las pendientes y las que no tienen pendiente
       ((df['Load'] == 0.5))] #carga 50% y las que su carga no está definida
pd.unique(df["Load"])
df = df.loc[(df["Mode"] == "Highway") | (df["Mode"].isnull()) | (df["Mode"] == "Urban Peak") | (df["Mode"] == "Hot")]
df["Mode"].unique()

array([nan, 'Urban Peak', 'Highway', 'Hot'], dtype=object)

In [28]:
#Cambiando clasificaciones
df.loc[df['Euro Standard'] == 'PRE ECE', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro 1', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro 2', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro 3', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro 4', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro 5', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro 6 a/b/c', 'Euro Standard'] = 'E6'
df.loc[df['Euro Standard'] == 'Euro 6 d', 'Euro Standard'] = 'E6'
df.loc[df['Euro Standard'] == 'Conventional', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro I', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro II', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro III', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro IV', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro V', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro VI A/B/C', 'Euro Standard'] = 'E6' #al parecer no hay diferencia en los parámetros estudiados
df.loc[df['Euro Standard'] == 'Euro VI D/E', 'Euro Standard'] = 'E6'
df.loc[df['Fuel'] == 'Petrol', 'Fuel'] = 'Gasolina'
#df.to_excel('C5_reducido_prueba.xlsx',index = False)

In [29]:
#Filtrando
df = df.loc[
    (df["Euro Standard"] == "No cat") |
    (df["Euro Standard"] == "E1") |
    (df["Euro Standard"] == "E2") |
    (df["Euro Standard"] == "E3") |
    (df["Euro Standard"] == "E4") |
    (df["Euro Standard"] == "E5") |
    (df["Euro Standard"] == "E6") |
    (df["Euro Standard"] == "E6")
]

In [30]:
#Filtrando tecnologias
df = df.loc[
            #Todo lo en blanco
            (df['Technology'].isnull()) |
            #Passenger Cars
            ((df['Category'] == 'Passenger Cars') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
            #Light Commercial Vehicles
            ((df['Category'] == 'Light Commercial Vehicles') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
             #Buses
            ((df['Category'] == 'Buses') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR')) |
            #Heavy Duty Trucks
            ((df['Category'] == 'Heavy Duty Trucks') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR'))
            ]

In [31]:
df2 = pd.read_excel("input/Velocidades.xlsx")


In [32]:
#Calculando parámetros para determinación de factores de emisión
bdf = df2.merge(df, left_on=["Modo", "Motorizacion", "Norma"], right_on=["Segment","Fuel","Euro Standard"], how="left")
alpha = bdf[["Alpha"]].astype("float")
beta = bdf[["Beta"]].astype("float")
gamma = bdf[["Gamma"]].astype("float")
delta = bdf[["Delta"]].astype("float")
epsilon = bdf[["Epsilon"]].astype("float")
zeta = bdf[["Zita"]].astype("float")
hta = bdf[["Hta"]].astype("float")
redFactor = bdf[["Reduction Factor [%]"]].astype("float")
bdf.columns
vel = bdf.iloc[:,7:len(df2.columns)]
#df2.loc[(df2["Modo"]=="VLP") & (df2["Norma"]=="E1") & (df2["Region"]=="R13")]
#bdf.loc[(bdf["Modo"]=="VLP") & (bdf["Norma"]=="E1") & (bdf["Region"]=="R13") & (bdf["Pollutant"]=="CO")]

In [34]:
#checkeo si supera la velocidad máxima o es inferior a la mínima
vel2 = np.where(vel.gt(bdf["Max Speed [km/h]"], axis=0),bdf[["Max Speed [km/h]"]],vel) 
vel2 = pd.DataFrame(vel2)
vel2.columns = vel.columns
#cálculo de factores de emisión
discriminante = (epsilon.values*vel2**2+zeta.values*vel2+hta.values)
EFs = np.where(discriminante == 0, 0, (1 - redFactor.values)*((alpha.values * vel2**2 + beta.values * vel2 
+ gamma.values + (delta.values/vel2)) / (discriminante)))
EFs = pd.DataFrame(EFs)
EFs.columns = vel.columns
EFs = pd.concat([bdf.iloc[:,[0,1,2,3,4,-20]],EFs],axis=1)
EFs = EFs.groupby(["Region","Ambito","Modo","Motorizacion",'Norma',"Pollutant"],as_index=False).mean()
#EFs.loc[(EFs["Modo"]=="BUS RIG") & (EFs["Region"]=="R13") & (EFs["Pollutant"]=="PM Exhaust")]
EFs["Pollutant"].unique()

array(['CH4', 'CO', 'EC', 'N2O', 'NH3', 'NOx', 'PM Exhaust',
       'SPN23 [#/kWh]', 'VOC', 'PM Exhaust [g/kWh]', 'SPN23'],
      dtype=object)

In [13]:
#Calculo CO2 y SOx
gasDensidad = 0.730 #Ton/m3
gasPcalor = 11200 #Kcal/kg 
dieselPcalor = 10900 #Kcal/kg 
dieselDensidad = 0.840 #ton/m3
tJouleAtCal = 2.39*10**(-1) #de joule a cal 
kCalLt = (gasDensidad*1000*gasPcalor*0.001)
kCalLt = (dieselDensidad*1000*dieselPcalor*0.001)
######## Fin Datos ##########
#Calculando CO2, SOx, km/lt
ECs_gas=EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Gasolina")].iloc[:,6:53]
ECs_diesel = EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Diesel")].iloc[:,6:53]
FCgas = (1000*ECs_gas*tJouleAtCal)/(gasPcalor/1000) #gr/km
FCdies = (1000*ECs_diesel*tJouleAtCal)/(dieselPcalor/1000) #gr/km
CO2_gas = 44.0011*(FCgas/(12.011+1.008*1.8))
CO2_dies = 44.0011*(FCdies/(12.011+1.008*2))
SOx_gas = 2*5*(10**-5)*FCgas
SOx_dies = 2*5*(10**-5)*FCdies
kmltGas = (FCgas/(gasDensidad*10**3))**-1
kmltDies = (FCdies/(dieselDensidad*10**3))**-1
namesG = EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Gasolina")].iloc[:,0:6]
namesD = EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Diesel")].iloc[:,0:6]
CO2 = pd.concat([pd.concat([namesG,CO2_gas],axis=1), pd.concat([namesD,CO2_dies],axis=1)],axis=0)
CO2["Pollutant"] = "CO2"
SOx = pd.concat([pd.concat([namesG,SOx_gas],axis=1), pd.concat([namesD,SOx_dies],axis=1)],axis=0)
SOx["Pollutant"] = "SOx"
kmLt = pd.concat([pd.concat([namesG,kmltGas],axis=1), pd.concat([namesD,kmltDies],axis=1)],axis=0)
kmLt["Pollutant"] = "[km/lt]"

In [14]:
#Pasando g/kWh PM to gr/km
consumo = EFs.loc[
    ((EFs.Modo.str.contains("CAM*")) | (EFs.Modo.str.contains("BUS*"))) & (EFs["Pollutant"] == "EC") & 
    ((EFs["Norma"] == "E5") | (EFs["Norma"] == "E6"))]
pme4e5 = EFs.loc[
    ((EFs.Modo.str.contains("CAM*")) | (EFs.Modo.str.contains("BUS*"))) & (EFs["Pollutant"] == "PM Exhaust [g/kWh]")]

pme4e5_2 = pme4e5.iloc[:,6:54]*consumo.iloc[:,6:54].values*0.278
namesPM = pme4e5.iloc[:,0:6]
pme4e5_3 = pd.concat([namesPM,pme4e5_2],axis= 1, ignore_index=False)
pme4e5_3["Pollutant"] = "PM Exhaust"

In [15]:
#Calculo N2O y NH3
kilometraje = pd.read_excel("input/kilometraje.xlsx")
xls = pd.ExcelFile("input/N2O_VLCVLP.xlsx")
dfN2OVL = pd.read_excel(xls,"0-30", index_col=None) #contenido de azufre (ppm) en el combustible
#dfN2OVL["N2O"] = (dfN2OVL["a"]*kilometraje+ dfN2OVL["b"])*dfN2OVL["Base [mg/km]"]/1000
km = kilometraje.loc[((kilometraje["Modo"]=="VLP") | (kilometraje["Modo"]=="VLC"))]
mkm = km.merge(dfN2OVL, left_on=["Modo", "Motorizacion", "Norma"], right_on=["Modo","Motorizacion","Norma"], how="left")
N2O = mkm.iloc[:,7:54].multiply(mkm["a"],axis="index").add(mkm["b"],axis=0).multiply(mkm["Base [mg/km]"],axis="index")/1000
N2O = pd.concat([mkm.iloc[:,0:6],N2O],axis=1)
N2O["Pollutant"]="N2O"
xls2 = pd.ExcelFile("input/NH3_VLCVLP.xlsx")
dfNH3VL = pd.read_excel(xls2,"0-30", index_col=None)
mkm = km.merge(dfNH3VL, left_on=["Modo", "Motorizacion", "Norma"], right_on=["Modo","Motorizacion","Norma"], how="left")
NH3 = mkm.iloc[:,7:54].multiply(mkm["a"],axis="index").add(mkm["b"],axis=0).multiply(mkm["Base [mg/km]"],axis="index")/1000
NH3 = pd.concat([mkm.iloc[:,0:6],NH3],axis=1)
NH3["Pollutant"]="NH3"
CO2["Region"].unique()

array(['R01', 'R02', 'R03', 'R04', 'R05', 'R06', 'R07', 'R08', 'R09',
       'R10', 'R11', 'R12', 'R14', 'R15', 'R16', 'RM'], dtype=object)

In [16]:
EF = pd.concat([EFs,CO2,SOx,kmLt,pme4e5_3,N2O,NH3],axis=0) 

In [17]:
#Calculating BC
df_bc = pd.read_excel('input/factores_bc.xlsx')
df_bc.sort_values("Modo")
pm_concat = EF.loc[ EF["Pollutant"]=="PM Exhaust"]

df3 = pm_concat.merge(df_bc,left_on=["Modo","Motorizacion","Norma"],right_on=["Modo","Motorizacion","Norma"],how="left")
#df3["BC_f"] = df3["2014"] * df3["BC"]
#df2
BC = df3.iloc[:,6:53].multiply(df3["BC"], axis="index")
BC = pd.concat([df3.iloc[:,0:6], BC], axis=1)
BC["Pollutant"]="BC"

In [18]:
#Calculating NO and NO2
df_nox = pd.read_excel('input/factores_NOx.xlsx')
df_nox.sort_values("Modo")
nox_concat = EF.loc[ EF["Pollutant"]=="NOx"]
df3 = nox_concat.merge(df_nox,left_on=["Modo","Motorizacion","Norma"],right_on=["Modo","Motorizacion","Norma"],how="left")
NO = df3.iloc[:,6:53].multiply(df3["NO %"], axis="index")/100
NO2 = df3.iloc[:,6:53].multiply(df3["NO2 %"], axis="index")/100
NO = pd.concat([df3.iloc[:,0:6], NO], axis=1)
NO["Pollutant"] = "NO"
NO2 = pd.concat([df3.iloc[:,0:6], NO2], axis=1)
NO2["Pollutant"] = "NO2"

In [21]:
#Especiation VOCs
df_vocs = pd.read_excel("input/GEIA_VOC.xlsx")
df_vocs.sort_values("Modo")
voc_concat = EF.loc[ EF["Pollutant"]=="VOC"]
df3 = voc_concat.merge(df_vocs,left_on=["Modo","Motorizacion","Norma"],right_on=["Modo","Motorizacion","Norma"],how="left")

Ethane = multiplier(EF,voc_concat,df3.columns[53],df_vocs)
Propane = multiplier(EF,voc_concat,df3.columns[54],df_vocs)
Butane = multiplier(EF,voc_concat,df3.columns[55],df_vocs)
Pentane = multiplier(EF,voc_concat,df3.columns[56],df_vocs)
Hexane = multiplier(EF,voc_concat,df3.columns[57],df_vocs)
Ethene = multiplier(EF,voc_concat,df3.columns[58],df_vocs)
Propene = multiplier(EF,voc_concat,df3.columns[59],df_vocs)
Ethyne = multiplier(EF,voc_concat,df3.columns[60],df_vocs)
Olefine = multiplier(EF,voc_concat,df3.columns[61],df_vocs)
Benzene = multiplier(EF,voc_concat,df3.columns[62],df_vocs)
Toluene = multiplier(EF,voc_concat,df3.columns[63],df_vocs)
Xylene = multiplier(EF,voc_concat,df3.columns[64],df_vocs)
Trimethylbenzene = multiplier(EF,voc_concat,df3.columns[65],df_vocs)
Other_aromatics = multiplier(EF,voc_concat,df3.columns[66],df_vocs)
Methanal = multiplier(EF,voc_concat,df3.columns[67],df_vocs)
Aldehyde = multiplier(EF,voc_concat,df3.columns[68],df_vocs)
Alkanone = multiplier(EF,voc_concat,df3.columns[69],df_vocs)
#Ethane.to_excel("Ethane.xlsx",index=False)
#voc_concat.iloc[:,6:].sum()
EF_VOCs=pd.concat([Ethane,Propane,Butane,Pentane,Hexane,Ethene,Propene,Ethyne,Olefine,Benzene,Toluene,Xylene,
                  Trimethylbenzene,Other_aromatics,Methanal,Aldehyde,Alkanone], axis=0)
EF_VOCs.to_excel("EF_VOCs_Species.xlsx", index=False)

In [20]:
#Formateando a lo STEP 3 y agregando Taxi/Motocicleta
EF = pd.concat([EFs,CO2,SOx,kmLt,pme4e5_3,N2O,NH3,BC,NO,NO2],axis=0) 
#Agregando Taxis
Taxi = EF.loc[(EF["Modo"]=="VLP") & (EF["Ambito"]=="Urbano")].copy()
Taxi.loc[Taxi["Modo"]=="VLP","Modo"] = "TAX"
EF = pd.concat([EF,Taxi], axis= 0)
#Agregando Motos E6
MotoE6 = EF.loc[(EF["Modo"]=="MOT") & (EF["Norma"]=="E5")].copy()
MotoE6["Norma"] = "E6"
EF = pd.concat([EF,MotoE6], axis= 0)
EF.insert(6, "Unidad", "g/km")
EF.loc[EF["Region"]=="RM", "Region"] = "R13"
EF.loc[EF["Pollutant"]=="PM Exhaust", "Pollutant"] = "PM"
#EF.rename(columns = {'Pollutant':'Gas'}, inplace=True)
EF = EF.sort_values(["Pollutant","Region","Modo","Ambito","Motorizacion","Norma"])
EF.to_excel("EF.xlsx",index=False)

,Region,Ambito,Modo,Motorizacion,Norma,Pollutant,Unidad,2014,2015,2016,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
20,R01,Interurbano,MOT,Gasolina,E1,BC,g/km,0.010000,0.010000,0.010000,...,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
21,R01,Interurbano,MOT,Gasolina,E2,BC,g/km,0.002500,0.002500,0.002500,...,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500
22,R01,Interurbano,MOT,Gasolina,E3,BC,g/km,0.002500,0.002500,0.002500,...,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500
23,R01,Interurbano,MOT,Gasolina,E4,BC,g/km,0.000438,0.000438,0.000438,...,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438
24,R01,Interurbano,MOT,Gasolina,E5,BC,g/km,0.000438,0.000438,0.000438,...,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438,0.000438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15775,R16,Urbano,MOT,Gasolina,E3,[km/lt],g/km,15.030789,15.030789,15.030789,...,15.030789,15.030789,15.030789,15.030789,15.030789,15.030789,15.030789,15.030789,15.030789,15.030789
15784,R16,Urbano,MOT,Gasolina,E4,[km/lt],g/km,18.858528,18.858528,18.858528,...,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528
15793,R16,Urbano,MOT,Gasolina,E5,[km/lt],g/km,18.858528,18.858528,18.858528,...,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528
15793,R16,Urbano,MOT,Gasolina,E6,[km/lt],g/km,18.858528,18.858528,18.858528,...,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528,18.858528
